# From Python to Production 
## Notebook 4 — Functions & Scope (LEGB Rule)
By **Prerna Joshi** | #25DaysOfDataTech  

"Functions turn chaos into structure — and scope explains why Python behaves the way it does."

---

### What you'll learn
- Function basics: definitions, returns, docstrings, annotations
- Parameters & arguments: positional, keyword, defaults, `*args`, `**kwargs`, unpacking
- Scope rules: **L**ocal, **E**nclosing, **G**lobal, **B**uilt-in (LEGB)
- `global` vs `nonlocal` — when (not) to use them
- Closures & decorator basics
- Common pitfalls (mutable defaults, shadowing) and production tips


> **Why this matters for data work**  
> Functions are your unit of reuse, testability, and composability. Understanding scope prevents accidental bugs, especially in notebooks and larger codebases.


## 1. Function Basics

A function packages logic and (optionally) returns a value. In production code, include a docstring and type hints for readability and tooling support.


In [1]:
def clean_name(raw: str) -> str:
    """Normalize a user name by trimming and title-casing.
    
    Args:
        raw: Original name (may include extra spaces or inconsistent casing).
    Returns:
        Normalized name, e.g., "  aLiCe  " -> "Alice".
    """
    return raw.strip().title()

clean_name("  aLiCe  ")


'Alice'

## 2. Parameters & Arguments

Python supports multiple parameter styles:
- **Positional** and **keyword** arguments
- **Defaults** (be careful with mutable defaults — see Section 6)
- **`*args`** for extra positional arguments
- **`**kwargs`** for extra keyword arguments
- **Unpacking** with `*` and `**` when calling


In [2]:
def summarize(order_id: int, status: str = "NEW", *items, priority=False, **meta):
    return {
        "order_id": order_id,
        "status": status,
        "items": items,        # tuple of extra positional args
        "priority": priority,  # keyword-only in this signature (after *items)
        "meta": meta,          # dict of extra keyword args
    }

summarize(101, "PENDING", "book", "pen", priority=True, source="mobile", user="pj")


{'order_id': 101,
 'status': 'PENDING',
 'items': ('book', 'pen'),
 'priority': True,
 'meta': {'source': 'mobile', 'user': 'pj'}}

## 3. Return Values

Functions can return multiple values (actually a tuple) and support early returns.


In [3]:
def min_max_avg(nums: list[float]) -> tuple[float, float, float]:
    n = len(nums)
    if n == 0:
        return float("nan"), float("nan"), float("nan")
    return min(nums), max(nums), sum(nums) / n

mn, mx, avg = min_max_avg([10, 20, 30, 40])
mn, mx, round(avg, 2)


(10, 40, 25.0)

## 4. Scope & The LEGB Rule

Python resolves names in this order:

```
L  Local        → names defined inside the current function/block
E  Enclosing    → names in any outer, non-global (enclosing) function scopes
G  Global       → names assigned at the module (file) level
B  Built-in     → names in Python's builtins module (e.g., len, print)
```

A quick mental model:

```
┌───────────────────────────────┐
│ Built-in (B)                  │  e.g., len, sum
├───────────────────────────────┤
│ Global (G)                    │  module-level variables
├───────────────────────────────┤
│ Enclosing (E)                 │  outer function variables (for closures)
├───────────────────────────────┤
│ Local (L)                     │  current function's variables
└───────────────────────────────┘
```


In [4]:
x = "global-x"   # G

def outer():
    x = "enclosing-x"  # E
    def inner():
        x = "local-x"  # L
        return x
    return inner()

outer()


'local-x'

## 5. Shadowing & Name Lookup

If a name is **assigned** inside a function, Python treats it as **local** by default. Reading a name before it's assigned locally raises `UnboundLocalError`.


In [5]:
y = 10
def f():
    # print(y)  # Uncommenting this line would raise UnboundLocalError
    y = 99
    return y

f()


99

## 6. Pitfall: Mutable Default Arguments

Default argument values are evaluated **once** at function definition time, not each call. Use `None` and create a new object inside.


In [6]:
def bad_append(item, bucket=[]):
    bucket.append(item)
    return bucket

def good_append(item, bucket=None):
    if bucket is None:
        bucket = []
    bucket.append(item)
    return bucket

bad1 = bad_append("a")
bad2 = bad_append("b")    # Surprise: reuses the same list
good1 = good_append("a")
good2 = good_append("b")

bad1, bad2, good1, good2


(['a', 'b'], ['a', 'b'], ['a'], ['b'])

## 7. `global` vs `nonlocal`

- Use **`global`** to rebind a module-level variable from inside a function (generally avoid in production code).
- Use **`nonlocal`** to rebind a variable from an enclosing (but non-global) scope — useful in closures.


In [7]:
counter = 0

def bump_global():
    global counter
    counter += 1
    return counter

def make_counter():
    c = 0
    def inc():
        nonlocal c
        c += 1
        return c
    return inc

bump_global(); bump_global()
inc = make_counter()
inc(), inc(), counter


(1, 2, 2)

## 8. Closures (Functions that remember)

A closure captures variables from an **enclosing** scope.


In [8]:
def make_multiplier(factor: int):
    def mul(x: int) -> int:
        return factor * x  # factor is captured from enclosing scope
    return mul

times3 = make_multiplier(3)
times3(7)


21

## 9. Decorators (Basics)

A **decorator** is a callable that takes a function and returns a function — great for cross-cutting concerns (timing, logging, caching).


In [9]:
import time
from functools import wraps

def timer(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        try:
            return fn(*args, **kwargs)
        finally:
            elapsed = (time.perf_counter() - start) * 1000
            print(f"{fn.__name__} took {elapsed:.2f} ms")
    return wrapper

@timer
def slow_add(a, b):
    time.sleep(0.05)
    return a + b

slow_add(3, 4)


slow_add took 50.54 ms


7

## 10. Lambdas & Functional Bits

Use small `lambda` functions for tiny transformations; prefer named functions for clarity.  
`map`, `filter`, and `sorted(key=...)` combine well with lambdas.


In [10]:
nums = [5, 2, 9, 1]
squared = list(map(lambda x: x*x, nums))
evens = list(filter(lambda x: x % 2 == 0, nums))
by_last_digit = sorted(nums, key=lambda x: x % 10)

squared, evens, by_last_digit


([25, 4, 81, 1], [2], [1, 2, 5, 9])

## 11. Docstrings & Type Hints

Type hints improve readability and enable static analysis (mypy, Pyright). Docstrings power tools like `help()` and IDE popups.


In [11]:
def read_scores(path: str) -> list[float]:
    """Read newline-separated floats from a file path."""
    scores: list[float] = []
    try:
        with open(path) as f:
            for line in f:
                line = line.strip()
                if line:
                    scores.append(float(line))
    except FileNotFoundError:
        # In a notebook/demo, just return a small default
        scores = [92.5, 88.0, 96.5]
    return scores

read_scores("scores.txt")


[92.5, 88.0, 96.5]

## 12. Practice (Try first, then reveal solutions below)

1. **`greet`**: Write a function `greet(name, title="")` that returns `"Hello, <Title> <Name>!"` (skip extra spaces if `title` is empty).  
2. **`safe_div`**: Create `safe_div(a, b, default=None)` that returns `a/b` or `default` if `b == 0`.  
3. **`flatten`**: Implement `flatten(nested)` that flattens a list like `[1,[2,3],[4,[5]]] → [1,2,3,4,5]`. Use recursion.  
4. **`make_tag`**: Write a closure `make_tag(tag)` that returns a function which wraps text like `"<h1>Title</h1>"`.  
5. **`once`**: Write a decorator `@once` that ensures a function runs only the first time; subsequent calls return the first result.  
6. **`sorted_by_length`**: Given a list of strings, return them sorted by length (ascending).  
7. **`accumulate`**: Build `accumulate(start=0)` that returns a closure `add(x)` which adds `x` to internal total and returns it (use `nonlocal`).  
8. **`arg_summary`**: Write `arg_summary(*args, **kwargs)` that returns a dict with counts and the actual args/kwargs.  
9. **`top_n`**: Implement `top_n(nums, n=3)` returning the `n` largest numbers without sorting the whole list (hint: `heapq`).  
10. **`safe_get`**: Write `safe_get(d, path, default=None)` where `path` is like `"a.b.c"` to navigate nested dicts.


## 13. Practice Solutions  
*(Click to reveal the answers only after solving.)*

<details>
<summary><strong>Solution 1️⃣ — greet</strong></summary>

```python
def greet(name, title=""):
    title = title.strip()
    return f"Hello, {title + ' ' if title else ''}{name}!"
```
</details>

<details>
<summary><strong>Solution 2️⃣ — safe_div</strong></summary>

```python
def safe_div(a, b, default=None):
    return a / b if b != 0 else default
```
</details>

<details>
<summary><strong>Solution 3️⃣ — flatten</strong></summary>

```python
def flatten(nested):
    out = []
    for x in nested:
        if isinstance(x, list):
            out.extend(flatten(x))
        else:
            out.append(x)
    return out
```
</details>

<details>
<summary><strong>Solution 4️⃣ — make_tag</strong></summary>

```python
def make_tag(tag):
    def wrap(text):
        return f"<{tag}>{text}</{tag}>"
    return wrap
```
</details>

<details>
<summary><strong>Solution 5️⃣ — once (decorator)</strong></summary>

```python
from functools import wraps

def once(fn):
    called = False
    result = None
    @wraps(fn)
    def wrapper(*args, **kwargs):
        nonlocal called, result
        if not called:
            result = fn(*args, **kwargs)
            called = True
        return result
    return wrapper
```
</details>

<details>
<summary><strong>Solution 6️⃣ — sorted_by_length</strong></summary>

```python
def sorted_by_length(strings):
    return sorted(strings, key=len)
```
</details>

<details>
<summary><strong>Solution 7️⃣ — accumulate (closure)</strong></summary>

```python
def accumulate(start=0):
    total = start
    def add(x):
        nonlocal total
        total += x
        return total
    return add
```
</details>

<details>
<summary><strong>Solution 8️⃣ — arg_summary</strong></summary>

```python
def arg_summary(*args, **kwargs):
    return {
        "num_args": len(args),
        "num_kwargs": len(kwargs),
        "args": args,
        "kwargs": kwargs
    }
```
</details>

<details>
<summary><strong>Solution 9️⃣ — top_n (heap)</strong></summary>

```python
import heapq

def top_n(nums, n=3):
    return heapq.nlargest(n, nums)
```
</details>

<details>
<summary><strong>Solution 🔟 — safe_get (dotted path)</strong></summary>

```python
def safe_get(d, path, default=None):
    cur = d
    for key in path.split("."):
        if isinstance(cur, dict) and key in cur:
            cur = cur[key]
        else:
            return default
    return cur
```
</details>


## 14. Cheatsheet

- **LEGB**: Local → Enclosing → Global → Built-in  
- Avoid `global` in production; prefer returning values or using objects
- For **mutable defaults**, use `None` then assign inside
- Use **closures** for stateful, minimal, testable helpers
- Decorators = function in, function out (use `functools.wraps`)
- Prefer **type hints** + clear docstrings for maintainability
